In [2]:
import cv2
from PIL import Image
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib as plt

In [53]:
base_dir = r"C:\Users\user\Documents\anaconda scripts\project 2.0\Face_Database"
train_dir=os.path.join(base_dir,"training_set")
validation_dir=os.path.join(base_dir,"cross_val_set")
names = os.listdir(train_dir)
print(names)

['arjun', 'dadi', 'karan', 'mummy']


In [54]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
                  train_dir,
                  target_size=(150,120),
                  batch_size=20,
                  color_mode="rgb",
                  class_mode='categorical')
validation_generator=validation_datagen.flow_from_directory(
                  validation_dir,
                  target_size=(150,120),
                  batch_size=16,
                  color_mode="rgb",
                  class_mode='categorical')

Found 1000 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [55]:
model = keras.Sequential([
    keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,120,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=4, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [56]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=6,
    verbose=2)

Epoch 1/10
50/50 - 16s - loss: 1.3222 - accuracy: 0.3810 - val_loss: 0.8952 - val_accuracy: 0.5000
Epoch 2/10
50/50 - 15s - loss: 0.8991 - accuracy: 0.6150 - val_loss: 0.4191 - val_accuracy: 0.8646
Epoch 3/10
50/50 - 15s - loss: 0.7373 - accuracy: 0.6990 - val_loss: 0.1619 - val_accuracy: 0.9583
Epoch 4/10
50/50 - 15s - loss: 0.4120 - accuracy: 0.8270 - val_loss: 0.3442 - val_accuracy: 0.8229
Epoch 5/10
50/50 - 15s - loss: 0.3028 - accuracy: 0.8790 - val_loss: 0.0310 - val_accuracy: 1.0000
Epoch 6/10
50/50 - 15s - loss: 0.2382 - accuracy: 0.9120 - val_loss: 0.0117 - val_accuracy: 1.0000
Epoch 7/10
50/50 - 15s - loss: 0.1335 - accuracy: 0.9500 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 8/10
50/50 - 16s - loss: 0.0763 - accuracy: 0.9700 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 9/10
50/50 - 15s - loss: 0.2092 - accuracy: 0.9220 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 10/10
50/50 - 16s - loss: 0.0535 - accuracy: 0.9850 - val_loss: 9.3041e-04 - val_accuracy: 1.0000


In [3]:
facedetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#img=cv2.imread(r"C:\Users\user\Pictures\Camera Roll\WIN_20200712_23_14_02_Pro.jpg")
img=cv2.imread(r"C:\Users\user\Documents\anaconda scripts\project 2.0\Face_Database\training_set\karan\karan.0.jpg")
cv2.imshow("face",img)
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces=facedetect.detectMultiScale(gray,1.1,5)
for(x,y,w,h)in faces:
    IMG = img[y-50:y+h+35,x-10:x+w+10]
    cv2.imshow("face",IMG)

In [57]:
facedetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

In [58]:
while(True):
    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray,1.1,5)
    for(x,y,w,h)in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        IMG = img[y-50:y+h+35,x-10:x+w+10]
        IMG = cv2.resize(img,(120,150))
        IMG = np.array(IMG)
        IMG=IMG/255
        IMG.resize((1,150,120,3))
        #pil_IMG = Image.fromarray(IMG)
        #pil_IMG.show()
        #pil_IMG=Image.fromarray(img[y-50:y+h+35,x-10:x+w+10])
        #pil_IMG.show()
        #IMG = pil_IMG.resize((150,120))
        #IMG = np.array(IMG)
        #IMG=IMG/255
        #IMG.resize((1,150,120,3))
        classes=model.predict(IMG)
        labels = list(classes[0])
        maxpos = labels.index(max(labels))
        name = names[maxpos]
        #print(maxpos)
        #print(labels[maxpos])
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(img,name,(x,y+h),font,1,(0,255,0),2)
    cv2.imshow("Face",img)
    if(cv2.waitKey(1)==ord('q')):
        break;

In [59]:
cap.release()
cv2.destroyAllWindows()